Brain states transitions in response to working memory training
==================================================================

Analyses performed by Karolina Finc & Justyna Kuk, 
*Centre for Modern interdisciplinary Technologies, Nicolaus Copernicus University in Toruń*

Last edited: 08-05-2019

--------------

The goal of this analysis is to examine the effects of working memory training on *time-resolved brain state dynamics* examined in the trained task and the resting-state. Does working memory training affects brain state distribution in the task? 

We used unsupervised machine learning approach to cluster dual n-back and resting-state fMRI time-series into discrete brain states (see Chen et al., 2015; Cornblath et al., 2018). We hypothesize that:
- The brain states fluctuation will differ after working memory training especially in the states related to default mode and frontoparietal systems activity. 
- Training-related changes in brain states dynamics will be more visible during the trained task than the resting state.
- Individual characteristics of brain states fluctuations will be associated to individual behavioral differences in training progress.


Step 1: Timeseries preparation
----------

Prior to running clustering the time-series into discrete brain stets, all timeseries were concatenated into large $N \times P$ array containing $N$ observation and $P$ features. The length of $N$ was equal to 227040 as a result of concatenating 4 sessions of dual n-back data (340 time-points) and resting state data (305 time-points) of 44 subjects. The length of $P$ was equal 400 and represented the mean signal extracted from each brain areas of Schaefer et al. (2019) brain parcellation.

By this procedure we ensured the correspondence of brain states labels across subjects, sessions and tasks.

(for now, we just testing code on small subsample of dataset -- 10 subjects)

In [5]:
#from nilearn import datasets
import numpy as np

n_sub = 46
# power = datasets.fetch_coords_power_2011()
# coords = np.vstack((power.rois['x'], power.rois['y'], power.rois['z'])).T
data = np.load("LB_dualnback_timeseries.npy")
data2 = np.load("LB_rest_timeseries.npy")

time_series_nback = data[0:n_sub]
time_series_rest = data2[0:n_sub]
print(time_series_nback.shape)
print(time_series_rest.shape)

(46, 4, 340, 264)
(46, 4, 305, 264)


In [8]:
#vectors of time-series:

t_nback = time_series_nback.reshape(n_sub,1360,264)    #for each subject
t_rest = time_series_rest.reshape(n_sub,1220,264)

t_vector_nback = time_series_nback.reshape(n_sub*4*340, 264)     #all 46 subcjects in one vector
t_vector_rest = time_series_rest.reshape(n_sub*4*305, 264)

In [13]:
#one vector for both rest and nback time-series:

t_vector = np.zeros((118680, 264))
t_vector[0:62560,:] = t_vector_nback
t_vector[62560:, :] = t_vector_rest

Step 2: Clustering the timeseries into brain states
----------------------

To discover main brain states existing in time-series we performed 500 repetitions of $k$-means clustering from $k$ = 2 to $k$ = 18 using Euclidean distance as a measure of similarity. 

In [5]:
from sklearn.cluster import KMeans

n_clusters = 17
group_k_labels = np.zeros((n_clusters,t_vector.shape[0]))

for k in range(n_clusters):
        kmeans = KMeans(n_clusters=k+2, n_init=100).fit(t_vector)
        k_labels = kmeans.labels_
        print(k_labels)
        group_k_labels[k,:] = k_labels


Step 3: Selecting k size based on silhouette score and absent states
-------------------------------------------------------------------------
To identify the optimal number of clusters we use silhouette score....

In [ ]:
from functions import number_of_clusters

number_of_clusters(8,t_vector_nback, n_init=500)  
number_of_clusters(8,t_vector_rest, n_init=500)  
number_of_clusters(8,t_vector, n_init=500)  

Step 4: Split-halves validation for k = X
-------------------------------------------------------------------------

We also split our sample into two equal partitions 500 times and performed k-means clustering separately on each half of the dataset. We then matched clusters by computing the correlation between both sets of centroids, and then by reordering the clusters based on the maximum correlation value for each cluster.

In [ ]:
from sklearn.model_selection import train_test_split

dim = int(t_vector.shape[0]/2)
train_series = np.zeros((500,dim,264))
test_series = np.zeros((500,dim,264))

for i in range(500):
    X_train, X_test = train_test_split(t_vector, test_size=0.5)
    train_series[i,:,:] = X_train
    test_series[i,:,:] = X_test

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
from scipy.stats import pearsonr

n_clusters = 6

cor = np.zeros((500,n_clusters,n_clusters))

for sub in range(500):
    train_kmeans = KMeans(n_clusters=n_clusters, n_init=500).fit(train_series[sub,:,:])
    test_kmeans = KMeans(n_clusters=n_clusters, n_init=500).fit(test_series[sub,:,:])
    x = train_kmeans.cluster_centers_
    y = test_kmeans.cluster_centers_
    for k in range(n_clusters):
        for j in range(n_clusters):
            cor[sub,k,j] = pearsonr(x[k],y[j])[0]

In [ ]:
cor_sort = np.sort(cor)
max_values = np.zeros((500,n_clusters))

for i in range(500):
    max_values = cor_sort[i,:,-1]
    mean_max_values = (cor_sort[i,:,-1]).mean


In [ ]:
from nilearn import plotting

num = 0

print(cor_sort[num,:,-1])
plotting.plot_matrix(cor_sort[num,:,:])

Step 5: Correlating cluster labels with well-known large-scale networks
-------------------------------------------------------------------------
We calculate corralation between mean of the time-series for each cluster and modules using Pearson correlation.

In [3]:
from functions import compare
compare(t_vector,n_clusters)

Step 5: Calculating persistence probablilities, transition probablilities, dwell times for each subject
-------------------------------------------------------------------------
Description

In [ ]:
#transition probabilities

from nilearn import plotting
import functions 

plotting.plot_matrix(functions.multiple_transition(n_clusters,t_rest).mean(axis=0), title = "rest")
plotting.plot_matrix(functions.multiple_transition(n_clusters,t_nback).mean(axis=0), title = "nback")

Step 6: Groups/sessions comparison
-------------------------------------------------------------------------
Description

In [ ]:
# Code here